### Example notebook to generate bactrack links from pre-existing masks (ie. Omnipose masks) using Scipy solver

In [15]:
from bactrack.widget import get_hierarchies_from_masks_folder
mask_dir = "/home/tlo/Documents/Data/bactrack_test4/xy0/masks"
hier_arr = get_hierarchies_from_masks_folder(mask_dir)

In [16]:
from bactrack.tracking import OverlapWeight, IOUWeight, DistanceWeight
w =  OverlapWeight(hier_arr)

2024-04-12 13:25:43,936 [INFO] Weight function start computing weight matrix
2024-04-12 13:25:43,956 [INFO] Weight matrix computed, time consuming:0.019055843353271484 sec


In [17]:
#   MIPSolver => python_mip repo: it includes two solvers: CBC and Gurobi
#   ScipySolver => scipy.optimize.milp: it include HiGHS solvers

# MIPSolver require conda environment and additional setup, so it not always avilable, you could use 
# ScipySolver instead
from bactrack.tracking import MIPSolver, ScipySolver

# do solver = ScipySolver(w.weight_matrix, hier_arr, mask_penalty= w.mask_penalty)
# if you want include a mask_penalty calculated by omnipose/cellpose mask threshold + flow threshold
solver = ScipySolver(w.weight_matrix, hier_arr)

# n is a list of all selected segementation
# e is a scipy sparse matrix mark the edge be selected
n, e = solver.solve()

In [18]:
from bactrack import io
# m is a mask array including all output masks
# e is a edge dataframe
# also selected cell in  hier_arr be marked in this function
m, e = io.format_output(hier_arr, n, e,overwrite_mask = False)
# now n is  a node dataframe
n = io.hiers_to_df(hier_arr)

In [19]:
# merge
import pandas as pd

merged_df = pd.merge(e, n.add_suffix('_source'), left_on='Source Index', right_on='index_source', how='left')
merged_df = pd.merge(merged_df, n.add_suffix('_target'), left_on='Target Index', right_on='index_target', how='left')

In [20]:
# export links to csv
import os

xydir = os.path.abspath(os.path.join(mask_dir, os.pardir)) #get xydir path
bactrackdir = os.path.join(xydir, "bactrackfiles")
os.makedirs(bactrackdir, exist_ok=True)  #make dir in xydir for bactrack file
merged_df.to_csv(os.path.join(bactrackdir, "mergedlinks.csv")) #save full merged links file 

selected_df = merged_df[['frame_source','label_source', 'label_target','area_source','area_target']]
selected_df.to_csv(os.path.join(bactrackdir, "superseggerlinks.csv")) #save links file relevant for supersegger